In [ ]:
%matplotlib inline  
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
from pyhdf import SD
import datetime
import mpl_toolkits.basemap
from mpl_toolkits.basemap import pyproj
import shapefile
import os,sys
from netCDF4 import Dataset
from numpy.random import uniform, seed
from matplotlib.mlab import griddata

In [ ]:
shpFilePath = '/home/bikascb/bikascb/MODIS_hdf/CountryBoundary/Outlinedd.shp'  
listx=[]
listy=[]
test = shapefile.Reader(shpFilePath)
for sr in test.shapeRecords():
    for xNew,yNew in sr.shape.points:
        listx.append(xNew)
        listy.append(yNew)
        
poly = mpl.path.Path(zip(listx,listy))

## Reading MODIS .hdf file and clip aod information for shapefile

In [ ]:
daily = {}
files = glob.glob('MOD04*')
files.sort() 
for f in files:
    #jdn = f[10:17]
    year = f[10:14]
    year = int(float(year))
    jdn = f[14:17] # julian day number
    jdn = int(float(jdn))
    Date = (datetime.datetime(year, 1, 1) + datetime.timedelta(jdn)).strftime("%Y%m%d")
    
    if daily.has_key(Date):
        daily[Date].append(f)
    else:
        daily[Date] = []
        daily[Date].append(f)


daily_values = []
for DATE in daily.keys():
    files = daily[DATE]
    
    latitude = np.array([])
    longitude= np.array([])
    aod = np.array([])

    
    for n in files:
        hdf=SD.SD(n)
        lat = (hdf.select('Latitude'))[:]
        lon = (hdf.select('Longitude'))[:]
        sds=hdf.select('Deep_Blue_Aerosol_Optical_Depth_550_Land_Best_Estimate')
        
        attrs = sds.attributes(full=1)
        scale_factor = attrs['scale_factor'][0]  
        fv = attrs['_FillValue'][0]
        
        data=sds.get()
        data=data.astype(float)
        
        data[data == fv] =np.nan
        data = np.ma.masked_array(data, np.isnan(data))
        data = data*scale_factor
        
        
        longitude = np.append(longitude,lon)
        latitude = np.append(latitude,lat)
        aod = np.append(aod,data)
        
        
        mask = poly.contains_points(zip(longitude,latitude))
        aod_ma = aod [mask]
        lat_ma = latitude [mask]
        lon_ma = longitude [mask]
        
    daily_data = {DATE: {'data': aod_ma, 'lon': lon_ma,'lat': lat_ma}}       
    for i in daily_data:
        daily_values.append(daily_data)



In [ ]:
dic = {}
for item in daily_values:
    key = item.keys()
    dic.update(item)

## Creating netcdf file 

In [ ]:
root_grp = Dataset('test.nc', 'w')
root_grp.description = 'AOD data of 10 km resolution'
dates = [] 
# dimensions
root_grp.createDimension('time', None)
root_grp.createDimension('lat', 50)
root_grp.createDimension('lon', 90)

# variables
times = root_grp.createVariable('time', 'f8', ('time',))
latitudes = root_grp.createVariable('latitude', 'f4', ('lat',))
longitudes = root_grp.createVariable('longitude', 'f4', ('lon',))
AOD = root_grp.createVariable('AOD', 'f4', ('time', 'lat', 'lon',))
AOD.fillvalue = '-9999'
AOD.offset = '0.00'
AOD.scalefactor = '0.0010000000474974513'
AOD.valid_range = '[0, 5000]'

#times.unit = 

# data
lats =  np.arange(26, 31, 0.1)
lons =  np.arange(80, 89, 0.1)
latitudes[:] = lats
longitudes[:] = lons

l = 0
for k in dic.keys():
    day = k
    
    times[l] = day
    l = l+1
#del x
#del y
#del z
j = 0
for i in dic.keys():  
    z = dic[i]['data']
    x = dic[i]['lon']
    y = dic[i]['lat']
    # define grid.
    xi = lons 
    yi = lats 
    # grid the data.
    zi = griddata(x, y, z, xi, yi, interp='linear')  
    #aod = zi.ravel()
    
    AOD[j] = zi
    j = j+1
    
root_grp.close()